In [1]:
#pandas and numpy are core libraries for data wrangling
import pandas as pd
import numpy as np

#re is not valuable for cleaning data that has been entered with inconsistent or incorrect syntax or type
import re

In [ ]:
#create a dataframe from lists or arrays
list1 = [1, 2, 3]
list2 = [2, 3, 4]
list3 = [3, 4, 5]
df = pd.DataFrame(np.array([list1, list2, list3]), columns=["column1", "column2", "column3"])

#another way to create a dataframe:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})

#create a dataframe from an existing dataframe piecemeal
df2 = pd.DataFrame(index = df.index)
df2["column 1"] = df["column1"]
df2["column 2"] = df["column3"]

#look at the header of the dataframe to see the structure
df2.head()

#source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

In [ ]:
#shows where values are missing in a dataframe
df.isna()

#counts missing values by column
df.isna().sum()

#drops columns with missing values from the dataframe
#use axis=1 to drop columns instead of rows.  
#how="any" drops rows where any of the checked values are missing, how="all" drops rows where all the checked values are missing
df.dropna(axis = 0, subset = ["list of columns to checked"], how = "any")

#interpolates to fill missing values
#use axis=1 to interpolate across rows rather than columns
df.interpolate(method="linear", axis = 0)

#interpolates, reverses the order of the data, interpolates again, and reverses the order back
#the point, here, is to catch missing values at both the end and the beginning of the data
df.interpolate().sort_index(ascending=False).interpolate().sort_index()

#Sources: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
#         https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html
#         https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html
#         https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_index.html

In [ ]:
#creates a list of unique values in a particular column in the dataframe
values = []
for i in df["column 1"]:
    if i not in values:
        values.append(i)
        
#another way to create lists of unique values
df["column name"].drop_duplicates().to_list()

#lists column names in a dataframe
df.columns

#renames columns in a dataframe
df.columns = ["column 1", "column 2", "column 3"]

#Sources: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
#         https://docs.scipy.org/doc/numpy/reference/generated/numpy.append.html

In [ ]:
#subset a dataframe and pick a list of values from the subset as a list
df[df["column 1"]==1]["column 2"]

#same thing, but as a list
list(df[df["column 1"]==1]["column 2"])

#examples of subsets with and (&), or (|), .isin(), and inequalities
target_list = [1,2]
df3 = df[(df["column 1"].isin(target_list))&(df["column 1"] > 1)]

#df.append adds rows to the bottom of a dataframe.  
df3.append(df[(df["column 2"].isin([2, 3]))|(df["column 3"] < 3)])

#drops selected columns from a dataframe
df3.drop(labels=["column 3"], axis=1)

#source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html
#        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [ ]:
#correct numeric data represented as strings with commas to the float datatype
#code checks for and skips over missing values, since re.sub() will throw an error if it hits them
nanlist = df.isna()["column 1"]
vallist = list(df["column 1"])
for i in range(len(vallist)):
    if nanlist[i] == False:
        vallist[i] = float(re.sub(",", "", vallist[i]))  

In [ ]:
#some rudimentary text scraping and cleaning
#pulls data out of text file, applies some cleaning with re.sub(), stores the result in another text file
#hits all the files in a directory -- note that the storage file is in a different directory from the target files
fw = open('storage_file.ext', 'w+')
os.chdir('filepath')
files = os.listdir()
for file in files:
    f = open(file)
    for line in f:
        if "target string" in line:
            line = re.sub("string to be corrected", "correction", line)
            fw.write(line)
    f.close()
fw.close()

#Source: me

In [ ]:
#create a dataframe of catagorical means by year
categories = []
for i in df["category column"]:
    if i not in categories:
        categories.append(i)
        
years = []
for i in df["year column"]:
    if i not in years:
        years.append(i)

columns_df = list(df.columns)
        
df_category_means = pd.DataFrame(columns=columns_df)
for category in categories:
    for year in years:
        df_hold = pd.DataFrame({"category column":[category],
                                "year":[year],
                                "mean_variable_of_interest": [np.mean(list(df[(df["category"] == category)&(df["year"] == year)]["variable_of_interest"]))]
                               })
        df_category_means = df_category_means.append(df_hold, ignore_index=True)
        
#Source: me

In [ ]:
#pivoting a dataframe
#note that this only works for a nice 2-dimensional table structure.  For higher dimensional data, adjustment will be required
df_pivot = df.pivot(index='column 1', columns='column 2')['column 3']

#source: https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.pivot.html



In [ ]:
#unpivoting, or "melting" a dataframe

df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})

pd.melt(df, id_vars=['A'], value_vars=['B'])

#source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html

In [9]:
#adding a calculated column to a dataframe
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})

#generate a list of values to be added
result_list = []
for i in range(len(df)):
    result_list.append(df["B"][i] + df["C"][i])

#turn the result list in to a one-column dataframe
add_df = pd.DataFrame({"Sum of B and C": result_list})

#join the new dataframe to the old one
df = df.join(add_df)

df


,A,B,C,Sum of B and C
0,a,1,2,3
1,b,3,4,7
2,c,5,6,11


In [ ]:
#Change a column in a dataframe to a new datatype:
df.column.astype(type)

#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html